<a href="https://colab.research.google.com/github/Interpause/pseudo-text/blob/master/testing_all_the_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#overview
I'm assuming everything in all-the-news is mostly true. It doesn't matter much for this model anyways since I am aiming to have it actively scrap relevant articles. I'll later figure out how to get adversarial examples (must indicate true news as fake when all given articles are fake; does this stop the model from picking on finer details like word usage?).

# Conclusion
This dataset is pretty bad. It will take a lot of cleaning to use.... I would still have used this datset though but I have found something better. Not exactly a dataset, but a modern newspaper scraper that can very accurately parse most mainstream news sources, including author, category, etc. I will be using that...

In [0]:
import sqlite3
import pandas as pd

In [2]:
#Online instance only.
from google.colab import drive #drive.flush_and_unmount()
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/CSIT Internship/code')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with sqlite3.connect('./raw/all-the-news.db') as db:
    raw = pd.read_sql_query('SELECT * from longform',db)

In [0]:
df = raw.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df = df.dropna(subset=['content'])
df = df[df.content.str.len() != 0]

In [0]:
df = df[df.content.str.len() >= 20]

In [0]:
remove = ('No.',',','[ ]',"['']",'[, ]','[] ,','“','[]')
def filterWeird(x):
    return all(not chk == x for chk in remove)
df = df[df.content.apply(filterWeird)]

In [39]:
len(df)

184835

In [45]:
set(df[df.content.str.len() < 100].content)

{'"GIRLS\' WATER POLO", \'CITY\', \'Championship, Saturday\', \'Eagle Rock 8, Palisades 1',
 '(Jane Bowles, “Everything is Nice”)',
 '(See also: Fathers and Sons.)',
 '/ nothing can parallel the very non-parallel fact of those pants and that floor',
 '/kaw·re?d?/:  Karen Gregory:  ReclaimUC: Bint Battuta:  @_kameelahr  \xa0 Kerim Friedman:',
 '/kaw·re?d?/:  Kerim Friedman:  Jacob Remes:  ReclaimUC:  Frank Pasquale:  Jacqui Shine:',
 '/kaw·re?d?/:  Kitabet:  Nathan Deuel:  Kerim Friedman:  Jacqui Shine: Reclaim UC:',
 '/kaw·re?d?/: Jacqui Shine:  Jacob Remes: ReclaimUC:  Bint Battuta:  Kerim Friedman:',
 '/kaw·re?d?/: Karen Gregory: Aishwarya Subramanian  Patrick Iber:  Reclaim UC: Kitabet:',
 '5 weeks. $2.\xa0Subscribe here.',
 'A bot-generated supercut of Donald Trump saying his own name.  By\xa0Sam Lavigne.',
 'A garden without its statue is like a sentence without its verb. Joseph W. Beach',
 'A stay at a medieval monastery often featured a lavish meal hot from the friar',
 'AUSTERI